In [1]:
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)
os.environ.get('OPENAI_API_KEY')

'sk-08GBKw9V6YPjQbuOojfI1hSWNSXO5LNKqczpX0EtbLTKnFZq'

In [2]:
from langchain_openai.chat_models import ChatOpenAI

In [3]:
chat = ChatOpenAI(
    openai_api_base=os.environ["CHATGPT_API_ENDPOINT"],
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

3.1 如何加載PDF和搜索網頁信息

In [4]:
from langchain.document_loaders import PyPDFLoader

In [5]:
##讀取PDF資料
loader = PyPDFLoader("./data/ML-guide.pdf")

In [6]:
pages = loader.load()

In [7]:
#確認頁數
len(pages)

22

In [8]:
#讀取第1頁前500字符訊息
page = pages[0]
print(page.page_content[:500])

MachineLearning-Lecture01  
Instructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine 
learning class. So what I wanna do today is just spend a little time going over the logistics 
of the class, and then we'll start to talk a bit about machine learning.  
By way of introduction, my name's Andrew Ng and I'll be instructor for this class. And so 
I personally work in machine learning, and I've worked on it for about 15 years now, and 
I actually think that machine learning is the 


In [9]:
#了解資料來源
page.metadata

{'source': './data/ML-guide.pdf', 'page': 0}

In [10]:
from langchain.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [11]:
#讀取網頁資料
loader = WebBaseLoader("http://google.com")

In [12]:
docs = loader.load()

In [13]:
print(docs[0].page_content)

Google搜尋 圖片 地圖 Play YouTube 新聞 Gmail 雲端硬碟 更多 »網頁記錄 | 設定 | 登入 進階搜尋Google 提供：  English 廣告商業解決方案關於 GoogleGoogle.com.tw© 2024 - 隱私權 - 服務條款  


In [14]:
#利用Serper搜尋訊息，並載入
import requests
import json

url = "https://google.serper.dev/news"

payload = json.dumps({
  "q": "apple inc",
  "hl": "zh-tw"
})
headers = {
  'X-API-KEY': 'b4a777328367a49d9e0c287a72c709995c66068f',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"searchParameters":{"q":"apple inc","hl":"zh-tw","type":"news","engine":"google"},"news":[{"title":"Is Apple Inc. (AAPL) the Best Kid-friendly Stock to Buy Right Now?","link":"https://finance.yahoo.com/news/apple-inc-aapl-best-kid-175848382.html","snippet":"We recently compiled a list of the 10 Best Kid-Friendly Stocks To Buy Right Now. In this article, we are going to take a look at where Apple...","date":"18 小時前","source":"Yahoo Finance","imageUrl":"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRqs3Ew-dTK28r3XdWgkrPRm-JtIU5tkqmRPSuzlUE1blx3MPEQ9CcevfvDkg&s","position":1},{"title":"Apple Inc. stock outperforms competitors on strong trading day","link":"https://www.marketwatch.com/data-news/apple-inc-stock-outperforms-competitors-on-strong-trading-day-d3d380d6-9bde2aa95ecc","snippet":"inched 0.32% higher to $259.02 Thursday, on what proved to be an all-around mixed trading session for the stock market, with the Dow Jones...","date":"2 天前","source":"MarketWatch","imageUrl":"htt

3.2 Text Splitter文本分割器

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=9,      #每個chunk大小(每幾個token切1組chunk)
    chunk_overlap=9    #前後chunk重疊的字符
)

In [17]:
text1 = "123456789"

In [18]:
text_splitter.split_text(text1)
#剛好9字符沒有分割

['123456789']

In [19]:
text2 = "123456789123456789"

In [20]:
text_splitter.split_text(text2)

['123456789',
 '234567891',
 '345678912',
 '456789123',
 '567891234',
 '678912345',
 '789123456',
 '891234567',
 '912345678',
 '123456789']

In [21]:
text3 = "This is a sample text to split. It has multiple sentences"

In [22]:
text_splitter.split_text(text3)

['This is a',
 'a sample',
 'text to',
 'split.',
 'It has',
 'multiple',
 'sentence',
 'sentences']

In [23]:
from langchain.document_loaders import PyPDFLoader

In [24]:
loader = PyPDFLoader("./data/ML-guide.pdf")
loader

In [25]:
pages = loader.load()
pages[0].page_content[:500]

"MachineLearning-Lecture01  \nInstructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine \nlearning class. So what I wanna do today is just spend a little time going over the logistics \nof the class, and then we'll start to talk a bit about machine learning.  \nBy way of introduction, my name's Andrew Ng and I'll be instructor for this class. And so \nI personally work in machine learning, and I've worked on it for about 15 years now, and \nI actually think that machine learning is the "

In [26]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]   #分割邏輯(有優先順序性)

)

In [27]:
len(pages)

22

In [28]:
docs = text_splitter.split_documents(pages)

In [29]:
#分割後，總共分成幾個chunk
len(docs)

172

In [30]:
#第1組chunk資料
docs[0].page_content

"MachineLearning-Lecture01  \nInstructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine \nlearning class. So what I wanna do today is just spend a little time going over the logistics \nof the class, and then we'll start to talk a bit about machine learning.  \nBy way of introduction, my name's Andrew Ng and I'll be instructor for this class. And so \nI personally work in machine learning, and I've worked on it for about 15 years now, and"

In [31]:
#第2組chunk資料
docs[1].page_content

"I personally work in machine learning, and I've worked on it for about 15 years now, and \nI actually think that machine learning is the most exciting field of all the computer \nsciences. So I'm actually always excited about teaching this class. Sometimes I actually \nthink that machine learning is not only the most exciting thing in computer science, but \nthe most exciting thing in all of human endeavor, so maybe a little bias there."

3.4 Chunking分塊大小怎麼決定

In [32]:
from langchain.document_loaders import ReadTheDocsLoader

In [36]:
#讀取html檔案，可把html的語法濾掉
loader = ReadTheDocsLoader(
    "htmldocs",   #可讀取該目錄下所有html文檔
    encoding="utf-8",  # 確保編碼與文件一致
    errors="ignore"    # 忽略無法解碼的字符
)     
docs = loader.load()

In [37]:
len(docs)

2

In [40]:
print(docs[0].page_content[:500])

langchain.indexes.vectorstore.VectorstoreIndexCreator¶
class langchain.indexes.vectorstore.VectorstoreIndexCreator[source]¶
Bases: BaseModel
Logic for creating indexes.
Create a new model by parsing and validating input data from keyword arguments.
Raises ValidationError if the input data cannot be parsed to form a valid model.
param embedding: Embeddings [Optional]¶
param text_splitter: TextSplitter [Optional]¶
param vectorstore_cls: Type[VectorStore] = <class 'langchain_community.vectorstores.


In [43]:
# gpt-3.5-turbo 4096 tokens
# If 4096 - (Input(Instruction + query + context) + output)
#     If Chunk nums = 5:
#         Chunk Size = 2000 / 5 = 400      *假設Instruction + query使用2000 tokens

# So Chunk Size <= 400

# Too small not meaningful
# Too big not efficient

In [45]:
import tiktoken

In [46]:
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
tokenizer

<Encoding 'cl100k_base'>

In [55]:
def token_count(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [56]:
tokens = [token_count(doc.page_content) for doc in docs]
tokens

[1538, 1605]

In [57]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [59]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=token_count,
    separators=["\n\n", "\n", " ", ""]
)

In [61]:
chunks = text_splitter.split_text(docs[0].page_content)
len(chunks)

5

In [62]:
token_count(chunks[0]), token_count(chunks[1]), token_count(chunks[2]), token_count(chunks[3]), token_count(chunks[4])

(383, 373, 345, 376, 105)

In [63]:
chunks

["langchain.indexes.vectorstore.VectorstoreIndexCreator¶\nclass langchain.indexes.vectorstore.VectorstoreIndexCreator[source]¶\nBases: BaseModel\nLogic for creating indexes.\nCreate a new model by parsing and validating input data from keyword arguments.\nRaises ValidationError if the input data cannot be parsed to form a valid model.\nparam embedding: Embeddings [Optional]¶\nparam text_splitter: TextSplitter [Optional]¶\nparam vectorstore_cls: Type[VectorStore] = <class 'langchain_community.vectorstores.inmemory.InMemoryVectorStore'>¶\nparam vectorstore_kwargs: dict [Optional]¶\nasync afrom_documents(documents: List[Document]) → VectorStoreIndexWrapper[source]¶\nCreate a vectorstore index from documents.\nParameters\ndocuments (List[Document]) – \nReturn type\nVectorStoreIndexWrapper\nasync afrom_loaders(loaders: List[BaseLoader]) → VectorStoreIndexWrapper[source]¶\nCreate a vectorstore index from loaders.\nParameters\nloaders (List[BaseLoader]) – \nReturn type\nVectorStoreIndexWrappe